# Chapter 4: Training Models

1. What linear regression training algorithm can you use if you have a training set with millions of features?

Probably mini-batch or stochasitc gradient descent would be good here, since neither needs to operate on all the features at once.

2. Suppose the features in your training set have very different scales. What algorithms might suffer from this, and how? What might you do about it?

Any gradient descent algorithm would suffer because certain features could skew the gradients simply by virtue of being on a different scale. This can be addressed by rescaling the features so that they are all on the same scale.

3. Can gradient descent get stuck in a local minimum when training a logistic regression model?

No, because the logistic regression cost function is convex.

4. Do all gradient descent algorithms lead to the same model provided you let them run long enough?

It depends on the learning rate. If this is set too high, the algorithm may never converge no matter how long it runs.
**NB** also if the cost function has local minima you could get stuck in one of those, which could lead to different solutions.

5. Suppose you use batch gradient descent and you plot the validation error at every epoch. If you notice that the validation error consistently goes up, what is likely going on? How can you fix this?

It's likely that the model is over-fitting to the training data. This can be addressed by regularizing and/or feeding the model more training data.

6. Is it a good idea to stop mini-batch gradient descent immediately when the validation error goes up?

Yes. Mini-batch should always be moving towards the optimal solution so if the validation error goes up that is a sign of over-fitting.

**WRONG**. These algorithms are stochastic and thus their validation error may jump around a bit. A better approach is to stop once validation error has not improved after some amount of time.

7. Which gradient descent algorithm (amongst those discussed) will reach the vicinity of the optimal solution fastest? Which will actually converge? How can you make the others converge as well?

Stochastic will reach the vicinity fastest but may not converge. Mini-batch will be a little slower but should converge. Stochastic can be made to converge by decreasing the learning rate as a function of the number of epochs.
**NB** batch gradient descent is also guaranteed to converge.

8. Suppose you are using polynomial regression. You plot the learning curves and you notice that there is a large gap between the training error and the validation error. What is happening? What are three ways to solve this?

This is likely indicative of over-fitting and can be addressed by choosing a simpler polynomial model, using some type of regularization, or re-shuffling the training/validation data to make sure the training data doesn't have some kind of bias.

9. Suppose you are using ridge regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularization hyperparameter alpha or reduce it?

I would say the model has high bias, since it seems that the model is not fitting the training or validation data well, suggesting the assumptions of the model do not align with the actual properties of the data. I would decrease alpha to allow the model to better fit the data.

10. Why would you want to use:

* Ridge regression instead of plain linear regression (i.e. without any regularization)?

This doesn't make sense, as ridge w/o regularization *IS* plain linear regression.

* Lasso instead of ridge regression?

Lasso is useful if you want to completely eliminate the weights of features that you think are unimportant.

* Elastic net instead of lasso?

You'd do this if you weren't sure whether you wanted to completely eliminate the unimportant features but did want to make sure they were down-weighted.
**NB** Lasso may behave erratically when some of the features are correlated and/or when there are many more features  than samples.

11. Suppose you want to classify pictures as outdoor/indoor and daytime/nighttime. Should you implement two logistic regression classifiers or one softmax regression classifier?

I'd say use one softmax regression classifier and have it distinguish b/w the 4 possible combinations of outdoor/day, outdoor/night, indoor/day, indoor/night.
**WRONG**. Kinda misunderstood the question. But the idea is that you don't care about distinguishing all 4 classes, just outdoor/indoor and daytime/nighttime. So in that case it makes more sense to train 2 logistic regression classifiers.

### Coding exercise
12. Implement batch gradient descent with early stopping for softmax regression (without using scikit-learn).

In [1]:
# load datasets and libraries
from sklearn import datasets
import numpy as np
iris = datasets.load_iris()

In [2]:
# get feature (X) and target (Y) vectors
X = iris.data
Y = iris.target

In [3]:
# set random seed so results are reproducible
np.random.seed(2042)

In [4]:
# function to split data into training and testing
def train_test_val_split(x, y, test_ratio=0.2, val_ratio=0.2):
    n = len(x) # how many items do we have?
    train_ratio = 1 - test_ratio - val_ratio
    train_n = int(n * train_ratio)
    test_n = int(n * test_ratio)
    # make a random permutation of indices from 0 to n
    idx = np.random.permutation(len(x))
    # split up the data
    train_idx = idx[:train_n]
    test_idx = idx[(train_n + 1):(train_n + 1 + test_n)]
    val_idx = idx[(train_n + 1 + test_n + 1):n]
    return x[train_idx], x[test_idx], x[val_idx], y[train_idx], y[test_idx], y[val_idx]

In [5]:
# function to convert class label vector into one-hot probablitiy encodings
def to_one_hot(x):
    nclasses = np.max(x) + 1
    a = np.zeros((len(x), nclasses))
    a[np.arange(len(x)), x] = 1
    return a

In [21]:
# function to compute the softmax of a vector of probabilities (aka logits)
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [7]:
# cross entropy cost function
def cost_function(p_hat, y, eps=1e-7):
    # p_hat is an array of logits
    # y is an array of booleans indicating the true class 
    m = p_hat.shape[0] # number of observations
    # it's ok to use regular multiplication (not matrix) here b/c y and p_hat will have the same shape
    return -np.mean(np.sum(y * np.log(p_hat + eps), axis=1))    

In [102]:
# just for shits, write my own dot product function
def my_dot(x, y):
    x = x.T
    res = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            res[i, j] = sum(x[i] * y[:,j])
    return res
## NB: turned out not to need this

In [124]:
# function to apply the model to a new set of test data
def apply_model(Theta, x):
    logits = x.dot(Theta)
    y_proba = softmax(logits)
    y_pred = np.argmax(y_proba, axis=1)
    return y_pred, y_proba

In [120]:
# function to train the softmax regressor using early stopping
def train_softmax(x, y, learning_rate=0.01, eps=1e-7, niter=int(1e3), early_stop_thresh=1e-3):
    # how many observations
    m = x.shape[0]
    # initialize the values for Theta, the matrix of feature weights (rows)
    # by possible classes (columns)
    Theta = np.random.randn(x.shape[1], y.shape[1])
    # start iterating
    for i in range(niter):
            # compute the logits
            logits = x.dot(Theta)
#             print("logits")
#             print(logits)
            # and the probabilities for each class
            y_proba = softmax(logits)
#             print("y")
#             print(y)
#             print("y_proba")
#             print(np.round(y_proba, 3))
            # value of the cost function
            loss = cost_function(y_proba, y, eps=eps)
#             print("loss")
#             print(loss)
            # what is the error of our predictions?
            error = y_proba - y
#             print("error")
#             print(np.round(error, 3))
#             print("x.T")
#             print(x.T)
            # some reporting
            if i % 500 == 0:
                print(i, loss)
            if loss <= early_stop_thresh:
                # reached our minimum loss tolerance, stop early
                print("Min. loss reached, stopping")
                return Theta
            # compute the gradients
            # NB: important to use / m instead of * (1/m), which resulted in matrix of all zeros
            # not sure why, might be some underlying issue w/ specific version of numpy
            gradients = x.T.dot(error) / m
#             gradients = my_dot(x, error)
#             foo = sum(x.T[0] * error[:,0])
#             print(foo)
#             print("gradients")
#             print(gradients)
            # update Theta
            Theta = Theta - learning_rate * gradients
    return Theta    

In [9]:
# need to add a bias term to every entry of X
X_with_bias = np.c_[np.ones([len(X), 1]), X]

# make the training, testing, validation datasets
X_train, X_test, X_val, Y_train, Y_test, Y_val = train_test_val_split(X_with_bias, Y, test_ratio=0.2, val_ratio=0.1)

# convert the class encodings to one-hot
Y_train_one_hot = to_one_hot(Y_train)
Y_test_one_hot = to_one_hot(Y_test)
Y_val_one_hot = to_one_hot(Y_val)

In [123]:
# train the model on the training data
model = train_softmax(X_train, Y_train_one_hot, niter=5000)

(0, 11.028748982128084)
(500, 0.57693039427387194)
(1000, 0.42377422768804718)
(1500, 0.34513035052375357)
(2000, 0.29377990732976739)
(2500, 0.25761576209088827)
(3000, 0.23084637006266792)
(3500, 0.21025189791355473)
(4000, 0.19391065818460784)
(4500, 0.18061431366846625)


In [127]:
# see how it performs on the validation data
y_val_pred, y_val_proba = apply_model(model, X_val)
print(y_val_pred)
print(Y_val)

[2 2 0 2 1 0 2 0 0 2 0 2 1]
[2 2 0 2 1 0 2 0 0 1 0 2 1]
